In [27]:
import numpy as np
import pandas as pd
import scipy.sparse
import xgboost as xgb
import mix_pandas as mix
import predict as predict_mix
import db_column_name as db

%matplotlib inline
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 10, 7


cn = db.ColumnName()

target_minT = pd.read_csv('./data/31286_103.csv')
mix.set_index_date(target_minT, cn.date)

X = pd.read_csv('./data/character_31286.csv')
mix.set_index_date(X, cn.date)

X = X.drop([cn.point], axis=1)
X = X[[x for x in X.columns if 'avg' in x or 
       x == cn.offset]]

X = X[X[cn.offset] == 69]
X = X[X.index.hour == 21]
print(X.shape)

# X = mix.mean_day(X)
# target_minT.index = target_minT.index.round('D')

X = X.drop([cn.offset], axis=1)

X = X.iloc[3:] # remove on change

target_minT = target_minT.reindex(X.index)
target_minT = mix.clean(target_minT)
X = X.reindex(target_minT.index)
X = mix.clean(X)
print(X.shape)

target_minT = mix.winsorized(target_minT, cn.value, [0.05, 0.95], 5)
X = X.reindex(target_minT.index)

(1245, 55)
(1192, 54)


In [32]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression, Lasso
from stepwise import stepwise_selection

selectColumns = stepwise_selection(X, target_minT)
X_select = X.loc[:, selectColumns]

X_select.head()

Add  avg t2                         with p-value 2.30822e-264
Add  avg rh2                        with p-value 2.22681e-56
Add  avg v850                       with p-value 4.69371e-10
Add  avg u500                       with p-value 0.00231836
Add  avg rot500                     with p-value 0.00957776


,avg t2,avg rh2,avg v850,avg u500,avg rot500
actual_date,,,,,
2014-06-20 21:00:00,16.79120,73.5068,-15.08000,-2.82198,-0.000025
2014-10-28 21:00:00,-15.38900,73.8041,-4.27446,-11.33590,0.000080
2014-10-30 21:00:00,-22.30960,100.0000,-10.26390,21.18470,-0.000053
2014-11-01 21:00:00,-9.84371,89.4535,2.18367,12.20310,0.000043
2014-11-02 21:00:00,-6.28352,92.6628,3.07034,5.90434,0.000040


In [33]:
predict = predict_mix.predict_model_split(LinearRegression(), X_select, target_minT, cn.value, 5)
for i, (train, test) in enumerate(predict):
    print("Train size {}".format(train.shape[0]))
    predict_mix.print_mean(train[[cn.value]], train[['prediction']])
    print("Test size {}".format(test.shape[0]))
    predict_mix.print_mean(test[[cn.value]], test[['prediction']])
    print(i)
    
# 16.3 8.4 16.5 12.3 15.3  

Train size 164
Mean squared error 12.96202
Mean absolute error 2.64366
Median absolute error 1.87312
Explained variance score 0.87778
Coefficient of determination 0.87778
Test size 163
Mean squared error 16.18719
Mean absolute error 3.24427
Median absolute error 2.84261
Explained variance score 0.92851
Coefficient of determination 0.92008
0
Train size 327
Mean squared error 13.23825
Mean absolute error 2.77780
Median absolute error 2.12382
Explained variance score 0.91549
Coefficient of determination 0.91549
Test size 163
Mean squared error 627.24019
Mean absolute error 20.83204
Median absolute error 20.27015
Explained variance score -1.65222
Coefficient of determination -4.23312
1
Train size 490
Mean squared error 44.07482
Mean absolute error 5.53000
Median absolute error 5.02905
Explained variance score 0.71901
Coefficient of determination 0.71901
Test size 163
Mean squared error 190.35868
Mean absolute error 12.38325
Median absolute error 12.46096
Explained variance score 0.48354
Co